In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import shutil

import wfdb
from wfdb import processing

%matplotlib inline

/home/singuyen/anaconda3/envs/biosignal/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import sys
sys.path.insert(0, '/home/singuyen/project_biosignal/c-labpl.qrs_detector.master')

from QRSDetectorOffline import QRSDetectorOffline

In [3]:
database = "incartdb"
database_dir = '../incartDb/'
record_ids = wfdb.get_record_list(database)

record_path = database_dir + record_ids[0]
signal_len = wfdb.rdrecord(record_path).__dict__['sig_len']

print(record_ids)
print(signal_len)

['I01', 'I02', 'I03', 'I04', 'I05', 'I06', 'I07', 'I08', 'I09', 'I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'I16', 'I17', 'I18', 'I19', 'I20', 'I21', 'I22', 'I23', 'I24', 'I25', 'I26', 'I27', 'I28', 'I29', 'I30', 'I31', 'I32', 'I33', 'I34', 'I35', 'I36', 'I37', 'I38', 'I39', 'I40', 'I41', 'I42', 'I43', 'I44', 'I45', 'I46', 'I47', 'I48', 'I49', 'I50', 'I51', 'I52', 'I53', 'I54', 'I55', 'I56', 'I57', 'I58', 'I59', 'I60', 'I61', 'I62', 'I63', 'I64', 'I65', 'I66', 'I67', 'I68', 'I69', 'I70', 'I71', 'I72', 'I73', 'I74', 'I75']
462600


# Run one sample

In [4]:
record_id = 'I01'
record_path = database_dir + record_id

#Take only 2 channels - I and V4
signal, fields = wfdb.rdsamp(record_path, channels=[0,9]) 
data = np.mean(signal, axis=1)
fs = fields['fs']
signal_len = fields['sig_len']

time = np.linspace(0, signal_len/fs, signal_len)
file = pd.DataFrame()
file['timestamp'] = time
file['ecg_measurement'] = data
file_name = '/tmp/' + database + '_' + record_id + '.csv'
file.to_csv(file_name, index=False)
    
qrs_detector = QRSDetectorOffline(ecg_data_path=file_name, fs=fs, 
                                  findpeaks_limit=0.1, findpeaks_spacing_factor=0.2,
                                  verbose=True,
                                  log_data=False, plot_data=False, show_plot=False)

detected_inds = qrs_detector.qrs_peaks_indices

qrs peaks indices
[   125    288    453 ... 462252 462426 462598]
noise peaks indices
[]


In [5]:
annotation = wfdb.rdann(record_path, 'atr')
sig, fields = wfdb.rdsamp(record_path, channels=[0,9])

# Compare detected qrs complexes to reference annotation.
comparitor = processing.compare_annotations(ref_sample = annotation.sample,
                                            test_sample = np.array(detected_inds),
                                            window_width = int(0.1 * fs),
                                            signal = sig)

# Print the results
comparitor.print_summary()

2757 reference annotations, 2414 test annotations

True Positives (matched samples): 2413
False Positives (unmatched test samples: 1
False Negatives (unmatched reference samples): 344

Specificity: 0.8752 (2413/2757)
Positive Predictivity: 0.9996 (2413/2414)
False Positive Rate: 0.0004 (1/2414)


True Negatives = Total # samples - Positives - False Negatives

In [6]:
comparitor.matched_ref_inds

array([   0,    1,    2, ..., 2754, 2755, 2756])

# Run all samples

In [18]:
ref_anno=[]
test_anno=[]

matched_ref_inds=[]
matched_test_inds=[]
unmatched_ref_inds=[]
unmatched_test_inds=[]

tp=[]
fp=[]
fn=[]

specificity=[]
pos_predictivity=[]
fpr=[]

for record_id in record_ids:
    record_path = database_dir + record_id

    #Take only 2 channels - I and V4
    signal, fields = wfdb.rdsamp(record_path, channels=[0,9]) 
    data = np.mean(signal, axis=1)
    fs = fields['fs']
    signal_len = fields['sig_len']

    time = np.linspace(0, signal_len/fs, signal_len)
    file = pd.DataFrame()
    file['timestamp'] = time
    file['ecg_measurement'] = data
    file_name = '/tmp/' + database + '_' + record_id + '.csv'
    file.to_csv(file_name, index=False)

    qrs_detector = QRSDetectorOffline(ecg_data_path=file_name, fs=fs, 
                                      findpeaks_limit=0.1, findpeaks_spacing_factor=0.2,
                                      verbose=True,
                                      log_data=False, plot_data=False, show_plot=False)

    detected_inds = qrs_detector.qrs_peaks_indices
    
    # Compare detected qrs complexes to reference annotation.
    annotation = wfdb.rdann(record_path, 'atr')
    ref_anno.append(annotation.sample)
    test_anno.append(np.array(detected_inds))
    
    if len(detected_inds) >= (len(annotation.sample)*0.8):
        comparitor = processing.compare_annotations(ref_sample = annotation.sample,
                                                    test_sample = np.array(detected_inds),
                                                    window_width = int(0.1 * fs),
                                                    signal = signal)

        # Store the results
        matched_ref_inds.append(comparitor.matched_ref_inds)
        matched_test_inds.append(comparitor.matched_test_inds)
        unmatched_ref_inds.append(comparitor.unmatched_ref_inds)
        unmatched_test_inds.append(comparitor.unmatched_test_inds)

        tp.append(comparitor.tp)
        fp.append(comparitor.fp)
        fn.append(comparitor.fn)

        specificity.append(comparitor.specificity)
        pos_predictivity.append(comparitor.positive_predictivity)
        fpr.append(comparitor.false_positive_rate)
    
    else:
        matched_ref_inds.append(np.nan)
        matched_test_inds.append(np.nan)
        unmatched_ref_inds.append(np.nan)
        unmatched_test_inds.append(np.nan)

        tp.append(np.nan)
        fp.append(np.nan)
        fn.append(np.nan)

        specificity.append(np.nan)
        pos_predictivity.append(np.nan)
        fpr.append(np.nan)

qrs peaks indices
[   125    288    453 ... 462252 462426 462598]
noise peaks indices
[]
qrs peaks indices
[    23    203    386 ... 462050 462264 462478]
noise peaks indices
[]
qrs peaks indices
[    19    179    351 ... 462098 462277 462454]
noise peaks indices
[]
qrs peaks indices
[    19    194    399 ... 462125 462307 462488]
noise peaks indices
[]
qrs peaks indices
[    95    353    619 ... 461829 462107 462381]
noise peaks indices
[]
qrs peaks indices
[   170    356    547 ... 440784 440962 456435]
noise peaks indices
[]
qrs peaks indices
[  3864   4233   4416 ... 403114 404946 405118]
noise peaks indices
[]
qrs peaks indices
[ 13262  22606  23385  23910  24462  25020  25584  26145  26689  27227
  27769  28582  29392  29921  30435  30958  32517  34948  39154  39406
  47049  51895  52041  52381  53695  54144  58278  59774  68629  72040
  72545  72795  77252  78546  78959  79616  79875  80118  82335  82605
  82872  83138  83678  85972  90503  91317  92168  93561  94358 126478
 126

qrs peaks indices
[    19    153    364    579    792   1001   1146   1404   1629   1855
   2075   2288   2504   2720   3112   3338   3556   3766   3970   4180
   4388   4596   4752   4912   5136   5354   5574   5788   6001   6207
   6413   6617   6825   7033   7239   7450   7662   7869   8078   8285
   8439   8641   8878   9081   9281   9482   9684   9883  10083  10273
  10463  10648  10832  11021  11214  11410  11611  11814  12017  12224
  12436  12643  12854  13067  13277  13496  13715  13930  14149  14364
  14582  14803  15020  15238  15458  15678  15892  16114  16333  16547
  16766  16984  17196  17417  17635  17845  18056  18897  19111  19329
  19545  19764  19982  20203  20420  20643  20864  21471  21680  21880
  22079  22276  22469  25541  31318  31706  31902  32094  32294  32497
  32702  32913  33278  33484  33676  33863  34009  34234  34422  34627
  34832  35035  35236  35442  35650  35859  36070  36664  36808  37049
  37252  37410  37590  37753  37978  38204  38428  38653  3

qrs peaks indices
[    20   1932  11006  13778  21315  24559  31019  33824  52297  53066
  56889  67920  74623  84973  96618  97233  97946  98817  99314  99814
 100996 101594 102280 104602 106296 106526 107176 108080 108308 108612
 108944 109172 109377 109598 110362 110907 111762 112199 112517 112840
 113698 114012 114889 115758 123473 123988 124778 125092 125410 129367
 129664 129964 130924 132572 132901 134132 134403 134684 134964 135246
 137945 138235 138518 138807 139094 139387 139674 139960 140248 140543
 140834 141122 141702 141998 142287 142590 142864 143149 146232 146543
 146845 174707 175811 178951 180685 183259 185568 191038 191718 199848
 201338 203020 203764 204999 207280 208101 209252 221169 226215 229469
 233640 237414 241268 248671 251641 253013 263443 264347 265057 265804
 266546 267227 267577 268913 269600 277943 278261 278555 279755 280930
 314053 322594 322948 323282 323609 324303 325002 334189 352709 355459
 357164 361036 361541 362818 364090 366082 367064 368727 37

qrs peaks indices
[128949 196684 381143 412827 415428 415624]
noise peaks indices
[]
qrs peaks indices
[ 64850 291594]
noise peaks indices
[]
qrs peaks indices
[    19  93542  96640 113760 135650 141462 144214 145220 158756 269101
 382768 387654]
noise peaks indices
[ 22697  39194  96515  97018  97530  97700 106803 107752 110407 111309
 113382 116174 119864 121362 123272 125213 127094 143757 144077 144488
 146152 155342 218635 218985 227955 228033 228097 228232 246145 248397
 250432 250868 253053 254347 256612 309470 311364 312062 319797 320375
 320520 323487 323568 324192 324791 325042 325514 359340 378472 379278
 386861 423965]
qrs peaks indices
[    19 380725 380992 387537 440940 441497 462122 462414]
noise peaks indices
[381931 382028 426168 426287 426348 426515 426802 427090 430235 430291
 432073 432511 433661 433771 433975 434053 434166 434372 434554 434810
 434960 435084 435735 436185 437679 439180 440199 441142 461787 461933]
qrs peaks indices
[   119    409    711 ... 461941 4

qrs peaks indices
[    26    714   1183   2816   3139   4025   4754   8380  10851  14540
  15094  15618  15782  17794  18007  18240  22324  22832  24547  24699
  27006  27316  27476  28107  28407  28696  28991  29768  30369  35824
  36639  39570  41719  42735  46153  47143  48305  50570  51674  51873
  52267  52456  52833  53016  53568  54780  56127  56444  57252  57408
  58038  58192  58638  58939  59089  59828  59975  60283  60451  60614
  60964  61404  61570  62326  62882  63215  63391  63713  63872  64022
  64494  64645  64798  65589  65911  66404  67032  67181  67766  68356
  68807  69129  69472  69820  70154  70473  70793  72491  72637  72785
  73936  74224  74365  74506  74786  75214  75358  75501  75645  75934
  76079  76223  76658  76799  76942  77369  77509  77651  77933  78075
  79073  79209  79768  80730  81925  82326  82729  83001  83554  84113
  85367  86613  86748  86886  87162  88390  88798  90050  90320  90731
  91421  91556  91699  91843  92841  92987  93446  93594  9

qrs peaks indices
[   368   6451   6862   7275   8655  12281  23883  32931  35773  47650
  64465  73711  74380  83466  85258  89889  93098  93989  94879 102917
 103707 107380 107791 119748 120133 121918 123194 126371 128096 130715
 131105 132806 134470 136261 137128 139655 140042 141794 142178 143913
 144300 145343 146096 146481 150030 151777 152166 153431 153819 154690
 155559 156791 157629 159358 160209 161075 164003 167251 168134 168624
 168993 171108 171503 172370 172764 174552 174955 176744 177149 180368
 180770 181172 181572 182498 183030 183372 185659 186076 187896 189713
 189959 190119 190365 190525 190758 190929 191150 191342 192702 193105
 193512 193916 194320 194557 194723 194949 195130 195534 195733 195935
 196139 196504 196759 196911 197165 197321 197567 197728 197966 198136
 198367 198546 198764 198970 199178 199382 199586 199793 200000 200354
 200636 200846 201056 201196 201473 201679 201882 202032 202295 202444
 202707 202855 203112 203264 203514 203673 203909 204072 20

In [19]:
summary = pd.DataFrame()
summary['record_id'] = record_ids

summary['ref_annotations'] = ref_anno
summary['test_annotations'] = test_anno

summary['matched_ref_inds'] = matched_ref_inds
summary['matched_test_inds'] = matched_test_inds
summary['unmatched_ref_inds'] = unmatched_ref_inds
summary['unmatched_test_inds'] = unmatched_test_inds

summary['true_positives'] = tp
summary['false_positives'] = fp
summary['false_negatives'] = fn

summary['specificity'] = specificity
summary['positive_predictivity'] = pos_predictivity
summary['fpr'] = fpr

In [20]:
summary.head()

,record_id,ref_annotations,test_annotations,matched_ref_inds,matched_test_inds,unmatched_ref_inds,unmatched_test_inds,true_positives,false_positives,false_negatives,specificity,positive_predictivity,fpr
0,I01,"[114, 277, 442, 608, 710, 941, 1106, 1269, 143...","[125, 288, 453, 617, 951, 1116, 1280, 1442, 16...","[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[4, 16, 32, 52, 67, 100, 108, 113, 118, 139, 1...",[2413],2413.0,1.0,344.0,0.875227,0.999586,0.000414
1,I02,"[11, 192, 375, 487, 586, 746, 931, 1119, 1305,...","[23, 203, 386, 596, 757, 942, 1129, 1316, 1526...","[0, 1, 2, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[3, 9, 16, 23, 36, 41, 47, 54, 61, 69, 76, 81,...",[],2444.0,0.0,230.0,0.913987,1.000000,0.000000
2,I03,"[0, 172, 344, 519, 698, 878, 1059, 1243, 1430,...","[19, 179, 351, 526, 705, 885, 1066, 1250, 1437...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[143, 145, 147, 149, 151, 153, 155, 157, 159, ...","[1409, 1466]",2366.0,2.0,86.0,0.964927,0.999155,0.000845
3,I04,"[-11, 187, 393, 611, 834, 1059, 1292, 1529, 17...","[19, 194, 399, 617, 841, 1066, 1298, 1535, 176...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[0, 57, 168, 195, 367, 452, 492, 728, 729, 752...",[0],2349.0,1.0,75.0,0.969059,0.999574,0.000426
4,I05,"[88, 346, 612, 873, 1142, 1419, 1692, 1969, 22...","[95, 353, 619, 880, 1149, 1425, 1699, 1976, 22...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[35, 104, 148, 149, 173, 198, 228, 253, 295, 3...",[],1612.0,0.0,164.0,0.907658,1.000000,0.000000


In [22]:
summary[['specificity', 'positive_predictivity', 'fpr']].describe()

,specificity,positive_predictivity,fpr
count,21.000000,21.000000,21.000000
mean,0.945812,0.999784,0.000216
std,0.051958,0.000307,0.000307
min,0.808829,0.999155,0.000000
25%,0.907658,0.999576,0.000000
50%,0.964927,1.000000,0.000000
75%,0.992551,1.000000,0.000424
max,1.000000,1.000000,0.000845


In [23]:
summary.to_csv("incartDb_summary.limit0.1_spacing0.2.csv", sep="\t", index=False)